#**Import Libraries**

In [1]:
import tensorflow as tf
from keras.layers import Conv2D, BatchNormalization, MaxPooling2D,Flatten, Dense, Dropout
from keras.models import Sequential
import keras.applications
from keras.applications.efficientnet import preprocess_input, decode_predictions
from keras.preprocessing import image
import numpy as np
import matplotlib.pyplot as plt
import os
from keras.callbacks import Callback, EarlyStopping, ReduceLROnPlateau
from keras.optimizers import Adam

#**Required Functions**

In [2]:
def preprocessing(addr):
  # addr (input) is directory of the image
  img = keras.utils.load_img(addr, target_size=(224, 224))
  # Turn to array of shape (224, 224, 3)
  img = keras.utils.img_to_array(img)
  # Expand array into (1, 224, 224, 3)
  img = np.expand_dims(img, 0)
  return img

In [3]:
def compile_model(x_train,y_train,x_valid,y_valid,num_epochs):  
  effnetB0=Sequential()
  effnetB0.add(keras.applications.EfficientNetB0(weights='imagenet', include_top=True))
  effnetB0.add(Dense(2,activation='softmax'))
  optimizer = Adam(lr=0.0001)

  early_stop = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=10, restore_best_weights=True)

  rlrop = ReduceLROnPlateau(monitor='val_loss', mode='min', patience= 5, factor= 0.5, min_lr= 1e-6, verbose=1)
  effnetB0.compile(optimizer=optimizer, loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits = True), metrics=['accuracy'])
  model_history = effnetB0.fit(x_train, y_train,validation_data = (x_valid, y_valid), callbacks = [early_stop, rlrop],verbose = 1, epochs = num_epochs)

In [4]:
def prediction(img,model,condition_check):
  pred = model.predict(img)
  if np.max(pred)*100<20 and condition_check:
    print('The Network could not classify the object.') 
    print('Maybe the object does not exist in the list of the labels!')
  else:
    for i in range(3):
      name = decode_predictions(pred)[0][i][1]
      prob = decode_predictions(pred)[0][i][2]*100
      print(f'Detected Object : {name}, Probability: {np.round(prob, 2)}%')

In [11]:
def Import_data(dir1,dir2):
  # Label of Computers = 1 , Label of Motorbikes = 0 
  x = []
  y = []
  img_folder=r'/content/'
  for file in os.listdir(os.path.join(img_folder, dir1)):       
    image_path1= os.path.join(img_folder, dir1,  file)
    x.append(preprocessing(image_path1))
    y.append(1)
  for file in os.listdir(os.path.join(img_folder, dir2)):
    image_path2=os.path.join(img_folder, dir2,  file)
    x.append(preprocessing(image_path2))
    y.append(0)
  x=np.array(x)
  x=x.reshape(len(y),224,224,3)
  y=np.array(y)
  randomize = np.arange(len(x))
  np.random.shuffle(randomize)
  x = x[randomize]
  y = y[randomize]
  return x,y

#**Part B**

In [6]:
effnetB0 = keras.applications.EfficientNetB0(weights='imagenet', include_top=True)

21834768/21834768 [==============================] - 0s 0us/step


In [7]:
addr=os.path.join(r'/content/','chair.jpg')
img = preprocessing(addr)
prediction(img,effnetB0,condition_check=False)

35363/35363 [==============================] - 0s 0us/step
Detected Object : rocking_chair, Probability: 24.9%
Detected Object : barrow, Probability: 7.75%
Detected Object : swing, Probability: 3.19%


#**Part C**

In [8]:
addr=os.path.join(r'/content/','bag.jpg')
bag=preprocessing(addr)
prediction(bag,effnetB0,condition_check=True)

1/1 [==============================] - 0s 120ms/step
The Network could not classify the object.
Maybe the object does not exist in the list of the labels!


#**Part D**

In [12]:
x_train,y_train=Import_data('C-train','M-train')

In [14]:
x_valid,y_valid=Import_data('C-valid','M-valid')

In [16]:
compile_model(x_train,y_train,x_valid,y_valid,5)

Epoch 1/5


/usr/local/lib/python3.8/dist-packages/keras/optimizers/optimizer_v2/adam.py:110: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)
/usr/local/lib/python3.8/dist-packages/tensorflow/python/util/dispatch.py:1082: UserWarning: "`sparse_categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a sigmoid or softmax activation and thus does not represent logits. Was this intended?"
  return dispatch_target(*args, **kwargs)


13/13 [==============================] - 142s 10s/step - loss: 0.6906 - accuracy: 0.8225 - val_loss: 0.6816 - val_accuracy: 0.7750 - lr: 1.0000e-04
Epoch 2/5
13/13 [==============================] - 123s 9s/step - loss: 0.6843 - accuracy: 0.9700 - val_loss: 0.6766 - val_accuracy: 0.8250 - lr: 1.0000e-04
Epoch 3/5
13/13 [==============================] - 124s 10s/step - loss: 0.6748 - accuracy: 0.9825 - val_loss: 0.6650 - val_accuracy: 0.9500 - lr: 1.0000e-04
Epoch 4/5
13/13 [==============================] - 122s 9s/step - loss: 0.6609 - accuracy: 0.9950 - val_loss: 0.6442 - val_accuracy: 1.0000 - lr: 1.0000e-04
Epoch 5/5
13/13 [==============================] - 125s 10s/step - loss: 0.6479 - accuracy: 0.9925 - val_loss: 0.6290 - val_accuracy: 1.0000 - lr: 1.0000e-04
